In [1]:
import pandas as pd 
import eurostat
df = eurostat.get_data_df('sbs_sc_sca_r2', flags=False)
df

,nace_r2,indic_sb,size_emp,geo\time,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
0,B,V11110,0-9,AT,NaN,183.0,192.0,215.0,226.0,230.0,227.0,238.0,238.0,242.0,245.0,244.0,237.0,NaN,NaN,NaN
1,B,V11110,0-9,BA,NaN,101.0,102.0,121.0,129.0,132.0,121.0,118.0,121.0,150.0,NaN,NaN,NaN,NaN,NaN,NaN
2,B,V11110,0-9,BE,NaN,140.0,118.0,173.0,144.0,167.0,149.0,175.0,160.0,227.0,249.0,248.0,NaN,NaN,NaN,NaN
3,B,V11110,0-9,BG,NaN,190.0,211.0,215.0,224.0,240.0,228.0,243.0,252.0,245.0,241.0,230.0,199.0,NaN,NaN,NaN
4,B,V11110,0-9,CH,NaN,82.0,84.0,81.0,85.0,84.0,86.0,89.0,75.0,75.0,79.0,85.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470233,S95,V92100,TOTAL,RS,NaN,4.4,4.3,4.2,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470234,S95,V92100,TOTAL,SE,NaN,1.1,1.1,1.2,1.2,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470235,S95,V92100,TOTAL,SI,NaN,1.5,1.5,1.5,1.5,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470236,S95,V92100,TOTAL,SK,NaN,1.5,1.4,1.5,1.4,1.4,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#eliminate columns for years previous to 2011
years =[]
for y in range(2005,2011): 
    years.append(y)
df=df.drop(years, axis=1)#drop columns with years before 2011
#rename geo time column to be easier to handle because of the name
df.rename(columns={"geo\\time": "geo"}, inplace=True)

In [3]:
#FUNCTION CALLABLE TO SELECT FOR EXAMPLE YEARS AND VARIABLE
import PySimpleGUI as sg
def selection(option):
    #sg.theme('DarkAmber')   # Add a touch of color
    #select the variable we want to calculate
    options = option

    # All the stuff inside your window.
    layout = [ 
            [sg.Text('Select one->'), sg.Listbox(options,select_mode=sg.LISTBOX_SELECT_MODE_SINGLE,size=(20,len(options)))],
            [sg.Button('Ok'), sg.Button('Cancel')]
        ]

    # Create the Window
    window = sg.Window('Make your choice', layout)

    # Event Loop to process "events" and get the "values" of the input
    while True:
        event, values = window.read()
        print( f"event={event}" )
        if event is None or event == 'Ok' or event == 'Cancel': # if user closes window or clicks cancel
            break
        
    # close  the window        
    window.close()

    if event == "Cancel":
        print( "You cancelled" )
    else:
        print('You entered ', values[0])
        sg.popup( f"You selected {values[0]}" )        
    return values

In [4]:
#selection of the variable (Calling the function)
dic_indic_sb = eurostat.get_dic('indic_sb') #dictionary for all the indicators in sbs
for indic in df.indic_sb.unique():
    if indic in dic_indic_sb:
        print (indic + " -"+dic_indic_sb[indic])

var=selection(df.indic_sb.unique())[0]
#determine the variable of interest based on the selection
for indic in var:
    if indic in dic_indic_sb:
        variable_of_interest=dic_indic_sb[indic]

print(variable_of_interest)
print(" ")


#create a list of years for selection
yearsloop =[]
for y in range(2011,2021): 
    yearsloop.append(y)
#select de year frame (Calling the function) with the starting year and the end year
year_start=selection(yearsloop)[0] #with [] we convert variable to a list to apply "isin"
year_end=selection(yearsloop)[0]
print(year_start)
print(year_end)

V11110 -Enterprises - number
V12110 -Turnover or gross premiums written - million euro
V12120 -Production value - million euro
V12150 -Value added at factor cost - million euro
V12170 -Gross operating surplus - million euro
V13110 -Total purchases of goods and services - million euro
V13310 -Personnel costs - million euro
V13320 -Wages and Salaries - million euro
V13330 -Social security costs - million euro
V16110 -Persons employed - number
V16120 -Unpaid persons employed - number
V16130 -Employees - number
V16150 -Hours worked by employees - number
V91100 -Turnover per person employed - thousand euro
V91110 -Apparent labour productivity (Gross value added per person employed) - thousand euro
V91120 -Wage adjusted labour productivity (Apparent labour productivity by average personnel costs) - percentage
V91130 -Gross value added per employee - thousand euro
V91150 -Gross value added per hour worked by employees
V91170 -Share of personnel costs in production - percentage
V91210 -Average

In [5]:
#select nace
nace_sel=['A', 'C13-C15','C17', 'C18', 'C20', 'C21', 'C24',
 'C25', 'C26', 'C27', 'C29', 'C30', 'C31', 'C32', 'E36', 'E37','E38','E39', 'J59_J60', 'J62_J63', 'M69_M70', 'M74_M75', 'Q87_Q88', 'R90-R92', 'T']

#countries to eliminate
countries_not_selected= ['UK', 'TR', 'AL','IS','LI','NO','CH','UK','ME','MK','AL','RS','TR','BA','EU28','EU15','EA','EA19','EA12', 'EU27_2007']
# columns of countries to be eliminated after sum
cols = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI','FR', 'HR', 'HU', 'IE', 'IT', 'LT','LU', 'LV', 'MT','NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']

#size total values (no distribution by size)
size_sel=['TOTAL']#we select the totals

In [6]:
import numpy as np
results_tot = pd.DataFrame() #create an empty dataframe for total results
#filter by size, nace, variable and countries
df_selection=df[df.size_emp.isin(size_sel)&df.nace_r2.isin(nace_sel)&df.indic_sb.isin(var)&~df.geo.isin(countries_not_selected)]
df_selection

,nace_r2,indic_sb,size_emp,geo,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
111993,C17,V11110,TOTAL,AT,129.0,128.0,128.0,138.0,135.0,140.0,141.0,137.0,141.0,142.0
111995,C17,V11110,TOTAL,BE,232.0,239.0,187.0,218.0,224.0,239.0,256.0,263.0,263.0,267.0
111996,C17,V11110,TOTAL,BG,505.0,495.0,496.0,498.0,512.0,511.0,488.0,506.0,496.0,507.0
111998,C17,V11110,TOTAL,CY,37.0,36.0,37.0,37.0,38.0,41.0,43.0,42.0,41.0,38.0
111999,C17,V11110,TOTAL,CZ,1027.0,1035.0,1003.0,986.0,975.0,950.0,928.0,895.0,929.0,958.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374419,E39,V11110,TOTAL,PT,23.0,16.0,15.0,13.0,16.0,19.0,21.0,26.0,24.0,27.0
374420,E39,V11110,TOTAL,RO,40.0,43.0,47.0,48.0,53.0,57.0,65.0,61.0,61.0,56.0
374422,E39,V11110,TOTAL,SE,122.0,119.0,112.0,129.0,123.0,109.0,114.0,116.0,117.0,113.0
374423,E39,V11110,TOTAL,SI,26.0,28.0,28.0,29.0,29.0,29.0,31.0,32.0,27.0,25.0


In [7]:
#iteration to retrieve the data for each year
for i in range (year_start[0],year_end[0]+1):
    results = pd.DataFrame() #create an empty dataframe for partial results
    df_target = pd.pivot_table(df_selection, values=i, index=['nace_r2'],columns=["geo"], aggfunc=np.sum, fill_value=0)#rearrange using nace
    df_target['sum'] = df_target[cols].sum(axis=1)#sums the columns of the countries to "sum"
    df_target=df_target.drop(cols, axis=1)#eliminate countries columns
    #now we can locate columns with 0 and make the substitution with the column "sum"
    df_target.loc[df_target['EU27_2020']==0,'EU27_2020'] = df_target['sum']#adds summ if column is empty
    moddf_target=df_target.drop('sum', axis=1)#drop sum column
    column_name=str(variable_of_interest)+str(i)#determine de column name using variable of interest and year
    results[column_name]= moddf_target['EU27_2020']
    results_tot = pd.concat([results_tot,results], axis=1)#adds column in each iteration (for each year)
results_tot

,Enterprises - number2011,Enterprises - number2012,Enterprises - number2013,Enterprises - number2014,Enterprises - number2015,Enterprises - number2016,Enterprises - number2017,Enterprises - number2018,Enterprises - number2019
nace_r2,,,,,,,,,
C17,18700,18561,18600,18700,17800,18400,17700,18340,18248
C18,112327,110322,111640,113244,107269,106939,103564,100000,105261
C20,25919,26019,25976,26334,27000,27000,27000,27986,28167
C21,3659,3600,3742,3605,3452,4088,3404,3452,3593
C24,17213,16204,15841,14981,14709,14708,14171,14000,14475
C25,368616,361136,352064,359334,357927,363776,362932,380000,384571
C26,36800,35900,36100,36000,34482,34471,34000,36417,36306
C27,48548,47484,46000,46000,43700,44000,42000,42350,42100
C29,17446,17200,16800,17300,16600,16800,16732,17000,17322


In [8]:
results_tot=results_tot.astype(float) #as originally data is integer need to change to float to compute division
results_tot.dtypes


Enterprises - number2011    float64
Enterprises - number2012    float64
Enterprises - number2013    float64
Enterprises - number2014    float64
Enterprises - number2015    float64
Enterprises - number2016    float64
Enterprises - number2017    float64
Enterprises - number2018    float64
Enterprises - number2019    float64
dtype: object

In [10]:
#create a dataframe for the ratios (copying the result_tot)
ratio_results =  results_tot.copy()
years= year_end[0]-year_start[0]
for i in range (0,years+1):
    for j in range (0,6):
    #17-18
        ratio_results.iloc[2*j][i]=results_tot.iloc[2*j][i]/(results_tot.iloc[2*j][i]+results_tot.iloc[(2*j)+1][i])
        ratio_results.iloc[(2*j)+1][i]=results_tot.iloc[(2*j)+1][i]/(results_tot.iloc[2*j][i]+results_tot.iloc[(2*j)+1][i])
    ratio_results.iloc[12][i]=results_tot.iloc[12][i]/(results_tot.iloc[12][i]+results_tot.iloc[13][i]+results_tot.iloc[14][i]+results_tot.iloc[15][i])
    ratio_results.iloc[13][i]=results_tot.iloc[13][i]/(results_tot.iloc[12][i]+results_tot.iloc[13][i]+results_tot.iloc[14][i]+results_tot.iloc[15][i])
    ratio_results.iloc[14][i]=results_tot.iloc[14][i]/(results_tot.iloc[12][i]+results_tot.iloc[13][i]+results_tot.iloc[14][i]+results_tot.iloc[15][i])
    ratio_results.iloc[15][i]=results_tot.iloc[15][i]/(results_tot.iloc[12][i]+results_tot.iloc[13][i]+results_tot.iloc[14][i]+results_tot.iloc[15][i])
ratio_results    
    #=ratio_tot.iloc["C31"][i]/(ratio_tot.iloc["C31", i]+ratio_tot.iloc["C32", i])


,Enterprises - number2011,Enterprises - number2012,Enterprises - number2013,Enterprises - number2014,Enterprises - number2015,Enterprises - number2016,Enterprises - number2017,Enterprises - number2018,Enterprises - number2019
nace_r2,,,,,,,,,
C17,0.142719,0.144014,0.142813,0.141727,0.142321,0.146802,0.145963,0.154977,0.147746
C18,0.857281,0.855986,0.857187,0.858273,0.857679,0.853198,0.854037,0.845023,0.852254
C20,0.876293,0.878456,0.874083,0.879588,0.886641,0.868502,0.888041,0.890197,0.886870
C21,0.123707,0.121544,0.125917,0.120412,0.113359,0.131498,0.111959,0.109803,0.113130
C24,0.044613,0.042943,0.043057,0.040022,0.039473,0.038860,0.037579,0.035533,0.036274
C25,0.955387,0.957057,0.956943,0.959978,0.960527,0.961140,0.962421,0.964467,0.963726
C26,0.431176,0.430538,0.439708,0.439024,0.441048,0.439283,0.447368,0.462338,0.463051
C27,0.568824,0.569462,0.560292,0.560976,0.558952,0.560717,0.552632,0.537662,0.536949
C29,0.579390,0.585392,0.585427,0.583907,0.570094,0.573379,0.564660,0.557615,0.557139


In [11]:
#save to file to calculate ration
fileratio="ratio-"+ variable_of_interest
filename = "%s.xlsx" % fileratio
ratio_results.to_excel(filename)